In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Improving-critical-exponents_pytorch/')
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
args_cnn = {
    'save_dir': './saved_models/cnn_regression',
    'lattice_size': 128,
    'dataset_size': 5000,
    'epochs': 100,
    'batch_size': 256,
    'dropout_rate': 0.0,
    'learning_rate': 10e-4,
    'device': device,
    'save_checkpoints': True,
}
        
args_cnn = objectview(args_cnn)

In [ ]:
from src.cnn.train import main as main_cnn
main_cnn(args_cnn)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1/100, Train Loss: 0.061908, Test Loss: 0.030898, Time: 19.74s
Epoch: 2/100, Train Loss: 0.003036, Test Loss: 0.011191, Time: 19.70s
Epoch: 3/100, Train Loss: 0.001858, Test Loss: 0.003371, Time: 18.01s
Epoch: 4/100, Train Loss: 0.001143, Test Loss: 0.000774, Time: 18.06s
Epoch: 5/100, Train Loss: 0.000916, Test Loss: 0.000783, Time: 18.09s
Epoch: 6/100, Train Loss: 0.000711, Test Loss: 0.000238, Time: 18.08s
Epoch: 7/100, Train Loss: 0.000354, Test Loss: 0.000182, Time: 18.06s
Epoch: 8/100, Train Loss: 0.000617, Test Loss: 0.000088, Time: 18.03s
Epoch: 9/100, Train Loss: 0.000623, Test Loss: 0.000502, Time: 18.05s
Epoch: 10/100, Train Loss: 0.000603, Test Loss: 0.000148, Time: 18.07s
Epoch: 11/100, Train Loss: 0.001037, Test Loss: 0.000600, Time: 17.99s
Epoch: 12/100, Train Loss: 0.000672, Test Loss: 0.000122, Time: 17.99s
Epoch: 13/100, Train Loss: 0.000489, Test Loss: 0.000069, Time: 18.06s
Epoch: 14/100, Train Loss: 0.000455, Test Loss: 0.000071, Time: 18.07s
Epoch: 15/100, 

In [ ]:
args_gen = {
    'save_dir': './saved_models/gan_cnn_regression',
    'epochs': 10,
    'batch_size': 5,
    'learning_rate': 10e-3,
    'device': device,
    'bins_number': 100,
    'noise_dim': 100,
    'ckpt_freq': 10,
    'CNN_model_path': './saved_models/cnn_regression/2021.10.22.08.09.39/model/final_model.pt',
    'set_generate_plots': False,
    'regularization_strength': 0.5,
}

args_gen = objectview(args_gen)

In [ ]:
from src.generator_cnn.train import main as main_gen
main_gen(args_gen)

In [5]:
args_hydra = {
    'lattice_size': 128,
    'dataset_size': 50,
    'batch_size': 10,
    'epochs': 2,
    'bins_number': 100,
    'learning_rate': 10e-3,
    'regularization_strength': 1.0,
    'discriminator_cnn_ratio': 1.0,
    'noise_dim': 100,
    'wanted_p': 0.5928,
    'save_dir': "./saved_models/hydra",
    'CNN_model_path': "./saved_models/cnn_regression/2021.10.22.08.09.39/model/ckpts/ckpt_89.pt",
    'device': "cpu",
    'set_generate_plots': False,
}

args_hydra = objectview(args_hydra)

In [6]:
from src.hydra.train import main as main_hydra
main_hydra(args_hydra)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: ignored